In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scorecardpy as sc
import time
import pickle
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
diretorio=os.getcwd()
diretorio

Uma vez que já temos o melhor modelo estimado com seus hyperparametros, voltamos ao pacote scorecardpy para aplicar esse resultado nos dados e construir os scores para cada individuo. Isso encerra esse ciclo de estudo para scorecard. Os melhoramentos que podem ser feitos aqui são de 3 tipos: i) novos dados; ii) tratamento dos dados; iii) diferentes estimadores.

### Modelo
Uma vez estimado o modelo final, podemos usar o mesmo no banco de dados para determinar os valores do score  para cliente.  Mas antes, vamos abrir o modelo que foi salvo.

In [43]:
with open("modelo_LR.pkl", 'rb') as file:  
    modelo = pickle.load(file)

modelo

LogisticRegression(C=0.001, penalty='l1', solver='liblinear')

### Dados
Os dados que usamos para fazer a parte final são os mesmos que usamos para fazer a investigação do woe e IV no notebook3. 

In [44]:
dataset = pd.read_csv('german_credit_data2.csv',index_col=0)
dataset['default']=np.where(dataset['default']=='yes',1,0)
dataset.head(2)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,default
0,67,male,2,own,little,little,1169,6,radio/TV,1
1,22,female,2,own,little,moderate,5951,48,radio/TV,0


Usamos a quantidade total de dados pois não há necessidade de separar entre treino e teste como na estimativa feita no notebook5 na escolha do melhor modelo. Agora nos ja temos o melhor modelo, e aplicamos seus coeficientes na estimativa do score para todos os dados.

## 1. scorecard
Para usar o modelo na construção dos scores, aplicamos duas funções: i) scorecard; ii) scorecard_ply. <br>
A função scorecard recebe como parametros importantes:
1. bins: o arquivo derivado do woebin;
2. model: especificação do modelo usado com os parâmetros estimados;
3. xcolumns: nome das colunas do banco de dados;
4. points0: ponto máximo para o score. Default=600. Caso queira um score que possa ir de 0-1000 mude o valor para 1000.
5. odds0: meta da chance p/(1-p)
6. pdo: valor para dobrar a chance. Default = 50;
7. basepoints_eq0: Default = False. Se for True, distribui o ponto base de forma igual para todas as variaveis;
8. digits: numero de digitos apos a casa decimal. Default=0.

#### 1.1. Os bins
Anteriormente aplicamos a função woebin para encontrar qual seria o melhor intervalo para cada variável utilizada. Podemos salvar esse arquivo e depois importar esse dicionário aqui. Abaixo temos esses valores:

In [ ]:
# %load woebin_20210616_230040.py
breaks_list={
'Age': [23.0,30.0,32.0,40.0,44.0,48.0,54.0], 
'Checking account': ['rich','moderate','little'], 
'Credit amount': [800.0,2400.0,3800.0,4800.0,6800.0,8800.0], 
'Duration': [10.0,14.0,18.0,22.0,30.0,44.0], 
'Housing': ['free','own','rent'], 
'Job': [2.0,3.0], 
'Purpose': ['domestic appliances%,%vacation/others%,%repairs%,%car','radio/TV','furniture/equipment','business%,%education'], 
'Saving accounts': ['little','quite rich','rich%,%moderate'], 
'Sex': ['female','male']}

Ou então replicar a função nos ddados para ter esse valor....

In [45]:
woe_bin=sc.woebin(dt=dataset, y='default',method="tree")
woe_bin

[INFO] creating woe binning ...


{'Credit amount':         variable              bin  count  count_distr  good  bad   badprob  \
 0  Credit amount     [-inf,800.0)     74        0.074    49   25  0.337838   
 1  Credit amount   [800.0,2400.0)    449        0.449   323  126  0.280624   
 2  Credit amount  [2400.0,3800.0)    203        0.203   133   70  0.344828   
 3  Credit amount  [3800.0,4800.0)     80        0.080    59   21  0.262500   
 4  Credit amount  [4800.0,6800.0)     81        0.081    66   15  0.185185   
 5  Credit amount  [6800.0,8800.0)     57        0.057    36   21  0.368421   
 6  Credit amount     [8800.0,inf)     56        0.056    41   15  0.267857   
 
         woe    bin_iv  total_iv  breaks  is_special_values  
 0  0.207914  0.003330  0.052153   800.0              False  
 1 -0.060512  0.001623  0.052153  2400.0              False  
 2  0.239004  0.012139  0.052153  3800.0              False  
 3 -0.152157  0.001792  0.052153  4800.0              False  
 4 -0.600746  0.025326  0.052153  6800.

#### 1.2. aplicando na função scorecard
De posse dos intervalos e do modelo, usamos a função scorecard para converter isso em um conjunto de informação para uso posterior. Essa é a única utilidade dessa função, unir o estimador com os intervalos, não se esquecendo de fornecer o nome das colunas do banco de dados. Veja que importamos a lista de intervalos que foi salva a partir do notebook3 onde aplicamos o pacote scorecardpy.

In [46]:
x=dataset.drop(['default'],axis=1)
card=sc.scorecard(bins='woebin_20210616_230040.py', 
                  xcolumns=x.columns, 
                  model=modelo, points0=1000)
card

{'basepoints':      variable  bin  points
 0  basepoints  NaN   788.0}

Ou entao usamos os resultados do woe_bin acima. Veja que usamos agora basepoints_eq0=False.

In [47]:
card2 = sc.scorecard(woe_bin, modelo, points0=1000,
                    xcolumns=woe_bin.keys())
card2

{'basepoints':      variable  bin  points
 0  basepoints  NaN   788.0}

o resultado da função "scorecard" por si não nos revela nada. Para converter os dados em score, temos que usar uma segunda função, a "scorecard_ply"....

## 2. scorecard_ply
Essa função calcular o score de credito usando os resultados da função "scorecard" e o banco de dados. Possui os seguintes parametros:
1. dt: Banco de dados original
2. card: output obtido da função 'scorecard'.
3. only_total_score: default = TRUE, incluindo somente o score total. Se for False, o output inclui alem do score total, o score para cada variavel.
4. print_step: Default = 1. Se print_step>0, mostra o nome das variáveis em cada iteração. Se print_step=0, nenhuma mensagem é mostrada.
5. replace_blank_na: Substitui os valores em branco por NA. Default = True.
6. var_kp: Determina o nome das colunas. Default = None.

In [52]:
score1 = sc.scorecard_ply(dataset, card2, only_total_score=True)